In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
pip install split-folders

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
import split_folders
import glob
import os
from PIL import Image
import matplotlib.pyplot as plt

from keras.models import model_from_json
import pickle

Using TensorFlow backend.


In [0]:
driveZip = '/content/drive/My Drive/temp/cropped_images.zip'
train_path = '/content/output/train/'
valid_path = '/content/output/val/'


image_types = ["kurti","none","saree","shirt","tshirt"]
batch_size = 32
image_size = [224,224]
num_classes = 4

In [0]:
import zipfile
with zipfile.ZipFile(driveZip,'r') as zip_dir:
    zip_dir.extractall(path='/content/')

In [0]:
split_folders.ratio('cropped_images', output="output", seed=1337, ratio= (.8,.2)) # default values

Copying files: 6383 files [00:02, 3143.71 files/s]


In [0]:
# Image size should be [224,224]
# Iterate through each color folder

def resize_images(image_dir):
  for im_type in image_types:
    # Iterate through each image file in each image_type folder
    #  glob reads in any image with the extension "image_dir/im_type/*"
    for file in glob.glob(os.path.join(image_dir, im_type, "*")):
        im = Image.open(file)
        f, e = os.path.splitext(file)
        imResize = im.resize((224,224), Image.ANTIALIAS)
        imResize.save(f + '.png', 'PNG', quality=90)

resize_images('/content/output/val')
resize_images('/content/output/train')

In [0]:
!rm -r NOA
!mkdir NOA

!mkdir NOA/train
!mkdir NOA/train/none
!cp -r '/content/output/train/none/' '/content/NOA/train/'
!rm -r /content/output/train/none

!mkdir NOA/val
!mkdir NOA/val/none
!cp -r '/content/output/val/none/' '/content/NOA/val/'
!rm -r /content/output/val/none

rm: cannot remove 'NOA': No such file or directory


In [0]:
!cp '/content/drive/My Drive/temp/VGG/secondLastModel.json' '/content/'
!cp '/content/drive/My Drive/temp/VGG/secondLastModel.h5' '/content/'

In [0]:
json_file = open('secondLastModel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

model.load_weights("secondLastModel.h5")
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [0]:
# tell the model what cost and optimization method to use
'''model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)'''

In [0]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [0]:
training_set = train_datagen.flow_from_directory('/content/output/train/',
                                                 target_size = (224, 224),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical')

'''training_set = train_datagen.flow_from_directory('/content/output/train/',
                                                 target_size = (224, 224),
                                                 batch_size = batch_size)'''

Found 4162 images belonging to 4 classes.


"training_set = train_datagen.flow_from_directory('/content/output/train/',\n                                                 target_size = (224, 224),\n                                                 batch_size = batch_size)"

In [0]:
predict = model.predict_generator(training_set,steps = len(training_set))

In [0]:
print(predict.shape)

(4162, 1000)


In [0]:
labels = np.full(predict.shape[0],1)
train1Set = np.concatenate((predict,np.asarray([labels]).T), axis=1)
print(train1Set.shape)

(4162, 1001)


In [0]:
#np.savetxt('train1Set.csv',train1Set, delimiter=',',fmt='%.15f')

In [0]:
training_set = train_datagen.flow_from_directory('/content/NOA/train/',target_size = (224, 224),
                                                 batch_size = batch_size)

Found 943 images belonging to 1 classes.


In [0]:
predict = model.predict_generator(training_set,steps = len(training_set))

In [0]:
labels = np.full(predict.shape[0],0,np.float32)
train0Set = np.concatenate((predict,np.asarray([labels]).T), axis=1)
print(train0Set.shape)

(943, 1001)


In [0]:
#np.savetxt('train0Set.csv',train1Set, delimiter=',',fmt='%.15f')

In [0]:
trainData = np.concatenate((train1Set,train0Set), axis=0)
print(trainData.shape)

(5105, 1001)


In [0]:
#np.savetxt('trainData.csv',trainData, delimiter=',',fmt='%.15f')

In [0]:
X = trainData[:,:-1]
Y = trainData[:,-1]
print(X.shape)
print(Y.shape)

(5105, 1000)
(5105,)


In [0]:
'''from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0,solver='saga',max_iter=5000)'''

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=5000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
from sklearn import svm
svm_c = 0.01
svm_gamma = 1
svm_kernal = 'rbf'
clf = svm.SVC(C=svm_c,kernel=svm_kernal,gamma=svm_gamma)

In [0]:
clf.fit(X, Y)

In [0]:
pickle.dump(clf, open('logRModel.sav', 'wb'))

In [0]:
!cp logRModel.sav '/content/drive/My Drive/temp/VGG'

# **Testing**

In [0]:
val_datagen = ImageDataGenerator(rescale = 1./255)

In [0]:
validation_set = val_datagen.flow_from_directory('/content/output/val/',
                                            target_size = (224, 224),
                                            batch_size = batch_size,
                                            class_mode = 'categorical')

Found 1042 images belonging to 4 classes.


In [0]:
predict = model.predict_generator(validation_set,steps = len(validation_set))

In [0]:
print(predict.shape)

(1042, 1000)


In [0]:
labels = np.full(predict.shape[0],1,np.float32)
test1Set = np.concatenate((predict,np.asarray([labels]).T), axis=1)
print(test1Set.shape)

(1042, 1001)


In [0]:
#np.savetxt('test1Set.csv',test1Set, delimiter=',',fmt='%.15f')

In [0]:
validation_set = val_datagen.flow_from_directory('/content/NOA/val/',
                                            target_size = (224, 224),
                                            batch_size = batch_size)

Found 236 images belonging to 1 classes.


In [0]:
predict = model.predict_generator(validation_set,steps = len(validation_set))

In [0]:
labels = np.full(predict.shape[0],0)
test0Set = np.concatenate((predict,np.asarray([labels]).T), axis=1)
print(test0Set.shape)

(236, 1001)


In [0]:
#np.savetxt('test0Set.csv',test0Set, delimiter=',',fmt='%.15f')

In [0]:
testData = np.concatenate((test1Set,test0Set), axis=0)
print(testData.shape)

(1278, 1001)


In [0]:
#np.savetxt('testData.csv',testData, delimiter=',',fmt='%.15f')

In [0]:
testX = testData[:,:-1]
testY = testData[:,-1]
print(testX.shape)
print(testY.shape)

(1278, 1000)
(1278,)


In [0]:
#clf = pickle.load(open('logRModel.sav', 'rb'))

In [0]:
acc = clf.score(testX,testY)

In [0]:
print(acc)